# Machine Translation Project 

This project aims to translate sentences from German to English. There are in total 200,000 pairs of sentences from IWSLT 2016. You can read more about the IWSLT 2016 data set [here](https://sites.google.com/site/iwsltevaluation2016/). The current model script is named as `transformer.py`, which contains a bert tokenizer and a transformer model. A transformer is a language model with attention mechanism that looks at an input sentence and decide which word of the sentence is important. If you are curious to learn more about what a transformer model is, feel free to check out this [blog post here](http://jalammar.github.io/illustrated-transformer/). The transformer model is implemented based on a tutorial linked [here](https://github.com/bentrevett/pytorch-seq2seq/blob/master/6%20-%20Attention%20is%20All%20You%20Need.ipynb). In this notebook, a transformer model will be used to generate translate German sentences to English.


This notebook aims to provide an example that highlights the ease of integrating with users' codes and the use of Azure Datastore. 

Table of Content:
1. [Set up workspace](#Set-up-workspace)
2. [Set up datastore](#Set-up-datastore)
3. [Specify compute target](#Specify-compute-target)
4. [Create an experiment](#Create-an-experiment)
5. [Create estimator and submit an experiment](#Create-estimator-and-submit-an-experiment)
6. [Results](#Results)
5. [Helpful to know: cancel runs](#Helpful-to-know:-cancel-runs)


Tip: if you need to debug your SDK, you can run this line `from azureml._logging.debug_mode import debug_sdk` and call `debug_sdk()` just before the code block you want to debug.

# Set up workspace

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import simplejson as json

import azureml.core
from azureml.core import Workspace, Experiment, Run, Datastore, ScriptRunConfig
from azureml.data.data_reference import DataReference

# import compute target 
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

from azureml.train.dnn import PyTorch

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.33


In [2]:
ws = Workspace.from_config()

In [3]:
# load config file

config_path = os.path.join(os.getcwd(), 'config.json')
print(config_path)
with open(config_path, 'r') as f:
    config = json.load(f)

/Users/chengyineng/Documents/Projects/microsoft-azure-ml-notebooks/machine-translation/config.json


In [4]:
print("This notebook was validated with the AzureML SDK version 1.0.33. You are currently using ", azureml.core.VERSION)

This notebook was validated with the AzureML SDK version 1.0.33. You are currently using  1.0.33


# Set up datastore

Here, we register a datastore specifically for this machine translation project. To read more about how to use a datastore to access your data, you can go to this [web page](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-access-data).

In [5]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)


# #define default datastore for current workspace
ds = Datastore.register_azure_blob_container(workspace=ws, 
                                             datastore_name='machine_translation', 
                                             container_name=config["container_name"],
                                             account_name=config["account_name"], 
                                             account_key=config["account_key"],
                                             create_if_not_exists=True)

#get named datastore from current workspace
ds = Datastore.get(ws, datastore_name='machine_translation')

AzureBlob amherstwstorageinnganzr azureml-blobstore-fe92660d-c6c1-4086-b2f7-71f9c508e6c7


In your local training script, which is `transformer.py` in this notebook, you can specify your data folder as such to mount the datastore on Azure:

```Python
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')
args = parser.parse_args()

data_folder = os.path.join(args.data_folder, 'machine_translation')
```

In [6]:
# Mount the datastore folder that you just created to find the data.
script_params = {
    '--data-folder': ds.as_mount()
}

As a sanity check, you can locate your datastore on the portal itself.
<br>
<img src="images/datastore.png" width="1500">

# Specify compute target

Specify a compute target based on an existing cluster or you can create a new compute target. You can view your existing clusters here 
<br>
<img src="images/compute_target.png" width="2000">

In [7]:
# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "nv12")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)
compute_target = ws.compute_targets[compute_name]
print(compute_target)

# Create an experiment

Create your experiment name within the workspace you desire.

In [8]:
experiment_name = 'machine_translation-transformer'
exp = Experiment(workspace=ws, name=experiment_name)

In the experiment tab, you should see the experiment that you just created on the portal.

<br>
<img src="images/experiment_name.png" width="1500">

# Create estimator and submit an experiment

In [9]:
pt_est = PyTorch(source_directory='./', 
                 script_params=script_params,
                 compute_target=compute_target,
                 entry_script='transformer.py',
                 pip_packages=['torchtext','bert-embedding'], # specify the packages that you need the run to install
                 use_gpu=True)

In [11]:
run = exp.submit(pt_est)

### To show widget of the experiment run details

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

<img src="images/widget.png" width="1500">

You can retrieve the logs for the experiment runs in the portal. The individual log files that you see on the portal keeps track of any `print` statements that you have included in your training script. 
<br>
<img src="images/log_details.png" width="1500">

Even if you have to submit multiple experiment runs, you do not have to worry about the changes that you make to the scripts because Azure keeps track of the scripts. You can even download a snapshot of your experiment run on the portal.
<br>
<img src="images/snapshot.png" width="1500">

## Results

After training the model, let's look at how our transformer model in German translation to English! 

***German Sentence (Source):***
<div style="background-color:rgba(0, 0, 0, 0.0470588); text-align:center; vertical-align: middle; padding:40px 0;" markdown="1"><span style="display:block" visibility="visible">
Aber wir hören wenig darüber, dass es da so etwas gibt wie Erwachsenen-Entwicklung und unsere 20er eine ausschlaggebende Zeit dafür sind. 
    </span>
</div>


***English Translation from Transfomer (Target)***: 
<div style="background-color:rgba(225, 225, 0, 0.3); text-align:center; vertical-align: middle; padding:40px 0;" markdown="1"><span style="display:block" visibility="visible">
But we hear little bit about that there's something like adults development and our 20s are a suggestion of time.
    </span>
</div>

***English Translation from Online Search Engine***:
<div style="background-color:rgba(0, 2250, 0, 0.04); text-align:center; vertical-align: middle; padding:40px 0;" markdown="1"><span style="display:block" visibility="visible">
But we hear little about the fact that there is such a thing as adult development and our 20s are a crucial time for it.
    </span>
</div>

We see that the transformer model is not _terrible_ and is able to translate long sentences pretty well (especially the first part!)

# Helpful to know: cancel runs

This function is useful when you realize you have made mistakes in your submitted training script (for example typos, if you are like me!). 

### To cancel the last run:

You can get the status of the submitted run. You can see that after running `.cancel()`, the experiment is now cancelled.

In [13]:
local_script_run = exp.submit(pt_est)
print("Did the run start?",local_script_run.get_status())

local_script_run.cancel()
print("Did the run cancel?",local_script_run.get_status())

Did the run start? Preparing
Did the run cancel? Canceled


### To cancel a run based on experiment ID

If you submitted multiple experiments and you want to cancel a specific experiment (because it is no longer relevant), you can retrieve an experiment's `Run Id` to cancel it directly. 

In [9]:
from azureml.core import get_run
run_cpu_id = 'machine_translation_1554915123_b8da5e9d' #get from portal
run=get_run(exp, run_cpu_id)
run.cancel()

Alternatively, you can also navigate to the experiment run on the portal to use the `cancel` button.
<br>
<img src="images/cancel_runs.png" width="800">